# Use namedtuple as Row

In [1]:
from collections import namedtuple
import sqlite3

User = namedtuple('User', ['uid', 'alias', 'shell'])

In [2]:
conn = sqlite3.connect(':memory:')
conn.execute('CREATE TABLE user (uid INT, alias TEXT, shell TEXT)')
conn.commit()

In [3]:
# Insert a few rows
rows = [
    (501, 'john', 'bash'),
    (502, 'karen', 'tcsh'),
]
conn.executemany('INSERT INTO user VALUES (?, ?, ?)', rows)
conn.commit()

In [4]:
# Without row factory, each row is returned as a tuple
for row in conn.execute('SELECT * FROM user'):
    print(row)

(501, 'john', 'bash')
(502, 'karen', 'tcsh')


In [5]:
# With a row factory, a custom type is returned for each row
conn.row_factory = lambda cursor, row: User._make(row)
for row in conn.execute('SELECT * FROM user'):
    print(row)

User(uid=501, alias='john', shell='bash')
User(uid=502, alias='karen', shell='tcsh')


In [7]:
# Use the built-in sqlite3.Row, we can index by int or by name
conn.row_factory = sqlite3.Row
for row in conn.execute('SELECT * FROM user'):
    print(f"uid={row['uid']}, alias={row[1]}, shell={row[2]}")

uid=501, alias=john, shell=bash
uid=502, alias=karen, shell=tcsh


In [18]:
# Dynamically create namedtuple, might now be as efficient
# as the sqlite3.Row

def make_namedtuple(cursor, row):
    labels = [tup[0] for tup in cursor.description]
    new_row = dict(zip(labels, row))
    blue_print = namedtuple("Row", labels)
    new_row = blue_print(*row)
    return new_row

conn.row_factory = make_namedtuple
for row in conn.execute('SELECT * FROM user'):
    print(row)

Row(uid=501, alias='john', shell='bash')
Row(uid=502, alias='karen', shell='tcsh')
